In [4]:
%reset -f
import pandas as pd
import numpy as np
import os
from wavhandler import *
import logging
import matplotlib.pyplot as plt
from scipy import signal

logger = logging.getLogger()
logger.setLevel(logging.WARN)

## Function to create the table
### It includes: evaluation of signals, extraction of top 3 frequencies and top 3 amplitudes

In [5]:
def make_table(wavhdlr):
    wavhdlr.read(create_table=True)
    df_tbl = wavhdlr.df_table
    evaluated = evaluate(df_tbl.names.tolist())
    
    df_tbl['evaluation'] = df_tbl['fname'].apply(lambda x: 1 if x in set(evaluated) else 0)

    sub1 = df_tbl.set_index('fname')
    psd_dict = psd_welch_freqs(sub1.names.tolist())
    sub2 = pd.DataFrame.from_dict(psd_dict, orient='index')

    ampl_dict = signal_amplitudes(sub1.names.tolist())    
    sub3 = pd.DataFrame.from_dict(ampl_dict, orient='index', columns=['ampl_1','ampl_2','ampl_3'])

    df = pd.concat([sub1, sub2, sub3], axis=1).reset_index(level='fname')
    return df

In [6]:
aedes = WavHandler('/home/yannis/data/insects/Potamitis/Wingbeats/Aedes', recursive=True)
anoph = WavHandler('/home/yannis/data/insects/Potamitis/Wingbeats/Anopheles', recursive=True)
culex = WavHandler('/home/yannis/data/insects/Potamitis/Wingbeats/Culex', recursive=True)

In [7]:
df_anoph = make_table(anoph)
df_anoph.to_csv('anoph_table.csv', sep=';')
del df_anoph

In [8]:
df_culex = make_table(culex)
df_culex.to_csv('culex_table.csv', sep=';')
del df_culex

In [9]:
df_aedes = make_table(aedes)
df_aedes.to_csv('aedes_table.csv', sep=';')
del df_aedes

## Reading back the csv files, combining them into 1 big dataframe and saving it as csv again

In [10]:
df_aedes = pd.read_csv('aedes_table.csv', delimiter=';')
df_anoph = pd.read_csv('anoph_table.csv', delimiter=';')
df_culex = pd.read_csv('culex_table.csv', delimiter=';')

In [11]:
big = pd.concat([df_aedes, df_anoph, df_culex], axis=0, sort=False)
big.to_csv('mosquitos.csv', sep=';')

In [ ]:
#[X] Fix the Fundamental and Harmonics : remove all these 31.25 values as fundamental, take into account the frequency value in peakseries, add it as column perhaps

#[X] Do PCA on PSD signal and find outliers

#[X] Separate Temperature and Humidity for signals that is available

#[X] Add 3 top amplitudes for signal

#[X] perhaps change the evaluate to remove butter and make separate preprosess function?